# Cleaning

In [1]:
import os
import pandas as pd
from tqdm import tqdm

import pathlib
from selectolax.parser import HTMLParser
from urllib.parse import parse_qs

Semua data yang berhasil di-*scrape* disimpan di folder `./raw`

In [2]:
len(os.listdir('raw'))

5113

Buat dataframe dan fungsi untuk menghasilkan+menambahkan data bersih dari setiap file ke dataframe

In [3]:
def get_href_param(a_tag):
    return parse_qs(a_tag.attributes.get('href').split('?')[1])

def formatted(filename):
    html = pathlib.Path(filename).read_text()
    tag_a = HTMLParser(html).css('a')

    return [
        {k:v[0] for (k,v) in get_href_param(a).items()}
        for a in tag_a
    ]


In [4]:
# len(formatted('raw/20100921.html'))

In [5]:
all_records = []
for fname in tqdm(sorted(os.listdir('raw'))):
    if fname[-4:]!='html':
        continue
    all_records.extend(formatted(f'raw/{fname}'))

100%|██████████| 5113/5113 [00:27<00:00, 184.46it/s] 


In [6]:
katalog = pd.DataFrame(all_records)

In [7]:
katalog

,tgl,ot,lat,lon,ketlat,ketlon,depth,mag,remark,status,strike1,dip1,rake1,strike2,dip2,rake2
0,2008/11/01,21:02:43.058,9.18,119.06,S,E,10,4.9,"Sumba Region, Indonesia",unset,NaN,NaN,NaN,NaN,NaN,NaN
1,2008/11/01,20:58:50.248,6.55,129.64,S,E,10,4.6,Banda Sea,unset,NaN,NaN,NaN,NaN,NaN,NaN
2,2008/11/01,17:43:12.941,7.01,106.63,S,E,121,3.7,"Java, Indonesia",unset,NaN,NaN,NaN,NaN,NaN,NaN
3,2008/11/01,16:24:14.755,3.30,127.85,S,E,10,3.2,"Seram, Indonesia",unset,NaN,NaN,NaN,NaN,NaN,NaN
4,2008/11/01,16:20:37.327,6.41,129.54,S,E,70,4.3,Banda Sea,unset,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95955,2022/10/31,05:51:22.667,2.90,128.69,S,E,10,2.6,Ceram Sea,unset,NaN,NaN,NaN,NaN,NaN,NaN
95956,2022/10/31,05:49:12.110,9.74,119.22,S,E,46,2.8,"Sumba Region, Indonesia",unset,NaN,NaN,NaN,NaN,NaN,NaN
95957,2022/10/31,03:05:58.313,7.77,106.85,S,E,51,4.7,"Java, Indonesia",unset,95.9,63.4,91.2,273.21,26.6,87.6
95958,2022/10/31,03:05:58.313,7.77,106.85,S,E,51,4.7,"Java, Indonesia",unset,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
for column in katalog:
    # mencari kolom dengan kurang dari 30 data unik
    z = set(katalog[column])
    if len(z)>30: continue

    print(column)
    print(z)

# ternyata bisa dihapus....

# katalog['ketlon'] sebenarnya hanya berisi 'E' dan 'W'
# Namun karena geografi Indonesia hanya berisi 'E'
# dan data selain 'E' hanya berisi data kotor... 
katalog = katalog[katalog['ketlon']=='E'] 

# bersihkan formatting data pada kolom remark
clean = lambda v: v\
    .replace('+', ' ')\
    .replace('%2C', '-')\
    .replace(', ', ' - ')\
    .strip()
katalog['remark'] = katalog['remark'].apply(clean)

# gabungkan data kolom ketlat dan kolom lat
# lat bernilai positif jika ketlat bernilai 'N'
# dan negatif jika ketlat bernilai 'S'
signed = lambda v: v[0] if v[1]=='N' else f'-{v[0]}'
katalog['lat'] = katalog[['lat', 'ketlat']].apply(signed, axis=1)

# hapus kolom-kolom yang tidak digunakan
katalog = katalog.drop(columns=['ketlon', 'ketlat', 'status'])
katalog = katalog.drop_duplicates()

katalog.to_csv('katalog_gempa.csv', index=False)

ketlat
{'S', 'N'}
ketlon
{'E'}
status
{'unset'}


In [ ]:
# katalog.fillna('')